In [1]:
from copy import deepcopy
import warnings

import numpy as np
import matplotlib.pyplot as plt
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
import openpyxl
import pandas as pd
from utils.hmatr import Hmatr

from utils.modelling import modellingSeriesStatistics
from utils.modelling import fixSeriesStatistics

%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

warnings.filterwarnings('ignore')
rpy2.robjects.numpy2ri.activate()
# utils = importr('utils')

# utils.chooseCRANmirror(ind=1)
# utils.install_packages('Rssa')

rssa = importr('Rssa')

D:\Program Files\anaconda\envs\researchWork\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [2]:
N = 700
w1 = 1/10
w2 = 1/5
C1 = 1
C2 = 2
phi1 = 0
phi2 = np.pi/2
Q = 301  # 301 номер, значит разладка в ряде будет на 302й точке, если ряд задан с 0.
B = 100
T_ = 100
L = 50
r = 2
noiseVariance = 0.5

method = "svd"

In [3]:
def plotSeries(s, title='Series', w=16, h=4):
    plt.figure(figsize=(w, h))
    plt.title(title)
    plt.plot(s)

Зададим наши функции

In [4]:
seriesPermanent = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C1*np.sin(2*np.pi*w2*n + phi1)
seriesTemporary = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C2*np.sin(2*np.pi*w1*n + phi1)
seriesShifted = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C1*np.sin(2*np.pi*w1*n + phi2)
seriesOutlier = lambda n: C1*np.sin(2*np.pi*w1*n + phi1)

Сгенерируем ряды с шумомо и без

In [5]:
np.random.seed(0)
eps = np.random.randn(N) * noiseVariance**2

fPerm = [seriesPermanent(i) for i in range(N)]
fPermNoise = fPerm + eps

fTemp = [seriesTemporary(i) for i in range(N)]
tmp = deepcopy(eps)
tmp[:Q] = tmp[:Q]/2
fTempNoise = fTemp + tmp

fShifted = [seriesShifted(i) for i in range(N)]
fShiftedNoise = fShifted + eps


fOutlier = [seriesOutlier(i) for i in range(N)]
fOutlier[Q] = fOutlier[Q] + C1*10
fOutlierNoise = fOutlier + eps

Промоделируем статистики шума

In [8]:
from utils.modelling import modellingNoiseStatistics

In [9]:
?modellingNoiseStatistics

Signature:
modellingNoiseStatistics(
    dictSeries:dict,
    iterNum:int,
    N:int,
    B:int,
    T:int,
    Q:int,
    L:int,
    r:int,
    method:str,
    vareps:float,
)
Docstring:
Моделирование статистик ряда (средний 95й процентиль и средний максимум) при различных реализациях шума до момента разладки методом Монте-Карло.
Внимание, шум добавляется внутри метода!
:param dict dictSeries: The dictionary where key is the type of series and value is a series. Example: { 'Permanent': [x_1, ..., x_N] }.
:param int iterNum: Number of iterations for modelling.
:param int N: The len of series.
:param int B: The len of base subseries.
:param int T: The len of test subseries.
:param int Q: The point of perturbation.
:param int L: The window len.
:param int r: Number of eigen vectors.
:param str method: SVD method.
:param float vareps: Variance of the noise.
:return: Pandas DataFrame
File:      f:\users\ivank\documents\education\magistracy\researchwork\researchwork\utils\modelling.py
Type:

In [21]:
%%time
iterNum = 200
dictSeries = dict(zip(['Permanent', 'Temporary', 'Shifted', 'Outlier'], [fPerm, fTemp, fShifted, fOutlier]))
statistics = modellingNoiseStatistics(dictSeries, iterNum, N, B, T_, Q, L, r, method, noiseVariance)
statistics.to_csv("tables/modelledStatistics.csv", index=False)

Wall time: 30min 40s


## Результаты моделирования

In [22]:
pd.read_csv("tables/modelledStatistics.csv")

,HeterType,StatType,row,col,sym,diag
0,Permanent,meanMax,0.132430,0.111197,0.129295,0.124817
1,Permanent,mean95Procentile,0.130548,0.110750,0.127285,0.123320
2,Temporary,meanMax,0.035480,0.030009,0.034706,0.033945
3,Temporary,mean95Procentile,0.034876,0.029890,0.034051,0.033554
4,Shifted,meanMax,0.133117,0.110919,0.130380,0.125972
5,Shifted,mean95Procentile,0.130551,0.110485,0.127657,0.124263
6,Outlier,meanMax,0.132626,0.111492,0.129723,0.126739
7,Outlier,mean95Procentile,0.130575,0.111044,0.127466,0.125366


## Промоделированные значения

In [27]:
%%time
modellingSeriesStatistics(
    dictSeries=dict(zip(['Permanent', 'Temporary', 'Shifted', 'Outlier'], [fPerm, fTemp, fShifted, fOutlier])),
    iterNum=2,
    N=N,
    B=B,
    T=T_,
    Q=Q,
    L=L,
    r=r,
    method=method,
    destFile='tables/results.xlsx',
    modellingResultsPath = 'tables/modelledStatistics.csv',
    title='withNoise',
    vareps=noiseVariance
)

Wall time: 17.7 s


In [41]:
fixedResult = pd.read_excel('tables/results.xlsx', sheet_name='withNoise', engine='openpyxl', usecols=range(18))
fixedResult.fillna(' ', inplace=True)
fixedResult[:40]

,Permanent,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Row,meanMax,95 procentile,,,Col,meanMax,95 procentile,,,Sym,meanMax,95 procentile,,,Diag,meanMax,95 procentile
1,Num Points of overcoming,2,2,,,Num Points of overcoming,2,2,,,Num Points of overcoming,2,2,,,Num Points of overcoming,2,2
2,Point of overcoming,318,318,,,Point of overcoming,338,338,,,Point of overcoming,319,319,,,Point of overcoming,318,318
3,X[Q],0.0334718,0.0334718,,,X[Q],0.0311036,0.0311036,,,X[Q],0.0320079,0.0320079,,,X[Q],0.0324622,0.0324622
4,X[Q+10],0.0720763,0.0720763,,,X[Q+10],0.0355709,0.0355709,,,X[Q+10],0.066485,0.066485,,,X[Q+10],0.0692073,0.0692073
5,X[Q+20],0.173986,0.173986,,,X[Q+20],0.0501011,0.0501011,,,X[Q+20],0.157122,0.157122,,,X[Q+20],0.168993,0.168993
6,X[Q+30],0.32168,0.32168,,,X[Q+30],0.0789346,0.0789346,,,X[Q+30],0.288597,0.288597,,,X[Q+30],0.315858,0.315858
7,,,,,,,,,,,,,,,,,,
8,,,,,,,,,,,,,,,,,,
9,Temporary,,,,,,,,,,,,,,,,,


In [73]:
def get_num_points_of_overcome(data):
    return {
        "Row": [data.iat[i, j] for i in range(1, 32, 10) for j in range(1, 3)],
        "Col": [data.iat[i, j] for i in range(1, 32, 10) for j in range(6, 8)],
        "Sym": [data.iat[i, j] for i in range(1, 32, 10) for j in range(11, 13)], 
        "Diag": [data.iat[i, j] for i in range(1, 32, 10) for j in range(16, 18)],
    }

def create_df_poc(data):
    res = get_num_points_of_overcome(data)
    ds = pd.DataFrame(columns=["HeterType", "StatType", "row", "col", "sym", "diag"])
    ds["HeterType"] = ['Permanent', 'Permanent', 'Temporary', 'Temporary', 'Shifted', 'Shifted', 'Outlier', 'Outlier']
    ds["StatType"] = ["meanMax", "mean95Procentile"]*4
    ds["row"] = res["Row"]
    ds["col"] = res["Col"]
    ds["sym"] = res["Sym"]
    ds["diag"] = res["Diag"]
    return ds

Выделим в отдельный датафрейм статистики по количеству преодолеваний функциями обнаружения неоднородности промоделированных значений шума.

In [74]:
fixedResult = pd.read_excel('tables/results.xlsx', sheet_name='withNoise', engine='openpyxl', usecols=range(18))
fixedResult.fillna(' ', inplace=True)
ds = create_df_poc(fixedResult)
ds

,HeterType,StatType,row,col,sym,diag
0,Permanent,meanMax,2,2,2,2
1,Permanent,mean95Procentile,2,2,2,2
2,Temporary,meanMax,2,2,2,2
3,Temporary,mean95Procentile,2,2,2,2
4,Shifted,meanMax,2,2,2,2
5,Shifted,mean95Procentile,2,2,2,2
6,Outlier,meanMax,2,0,2,2
7,Outlier,mean95Procentile,2,0,2,2


---

## Анализ разностей точек преодоления.

Проверим гипотезу о том, что чем больше разница в частотах ряда до и после возмущения, тем лучшее качество показывают функции обнаружения разладки.

In [ ]:
def generate_series(omega, N=700, Q=301):
    w1, w2 = omega
    seriesPermanent = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C1*np.sin(2*np.pi*w2*n + phi1)
    seriesTemporary = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C2*np.sin(2*np.pi*w1*n + phi1)
    seriesShifted = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C1*np.sin(2*np.pi*w1*n + phi2)
    seriesOutlier = lambda n: C1*np.sin(2*np.pi*w1*n + phi1)
    
    fPerm = [seriesPermanent(i) for i in range(N)]
    fTemp = [seriesTemporary(i) for i in range(N)]
    fShifted = [seriesShifted(i) for i in range(N)]
    fOutlier = [seriesOutlier(i) for i in range(N)]
    fOutlier[Q] = fOutlier[Q] + C1*10
    return dict(zip(['Permanent', 'Temporary', 'Shifted', 'Outlier'], [fPerm, fTemp, fShifted, fOutlier]))

In [ ]:
def start_testing(omegas, iterNum=200):
    for omega in omegas:
        modellingSeriesStatistics(
            dictSeries=generate_series(omega),
            iterNum=iterNum,
            N=N,
            B=B,
            T=T_,
            Q=Q,
            L=L,
            r=r,
            method=method,
            destFile='tables/resultsTesting.xlsx',
            modellingResultsPath = 'tables/modelledStatistics.csv',
            title=str(omega),
            vareps=noiseVariance
        )
        